# primary key 主键约束
将某一列变成主键，比如身份证号

一个表里只能有一个主键



In [ ]:
-- 新建一个有primary key的表
CREATE TABLE test_primary_key(
	id INT PRIMARY KEY,
	name VARCHAR(25)
);

-- 把某个列变成primary key，直接这么操作的话应该会报错
-- 因为前面已经设置好id为primary key
ALTER TABLE test_primary_key
ADD CONSTRAINT  PRIMARY KEY (name);

INSERT INTO test_primary_key
VALUES (1, 'hello'),
	   (2, 'world');


# 自动增长auto_increment
设置好以后，每次新增一个数据，primary key自动递增

In [ ]:
-- auto_increment 只能给primary key设置
CREATE TABLE test_auto_increment(
	id INT PRIMARY KEY AUTO_INCREMENT ,
	name VARCHAR(25)
);

-- 测试插入一些数据
INSERT INTO test_auto_increment (name)
VALUES ('first'),
	   ('second'),
	   ('third');

SELECT * FROM test_auto_increment;

-- 设置auto_increment的起始值从100开始，再插入试试看
ALTER TABLE test_auto_increment
AUTO_INCREMENT = 100;

# 外键 foreign key
一个表可以把一列数据关联为另一个表的primary key

In [ ]:
-- 先创建一个主要的表
CREATE TABLE test_foreign_key_main(
	id INT PRIMARY KEY AUTO_INCREMENT,
	first_name VARCHAR(50),
	last_name VARCHAR(50)
);

-- 写点数据
INSERT INTO test_foreign_key_main (first_name, last_name)
VALUES ('hello','world'), ('good','day');

-- 创建另一个表，并关联外键
CREATE TABLE test_foreign_key_ref(
	ref_id INT PRIMARY KEY AUTO_INCREMENT,
	amount DECIMAL(10,2),
	main_id INT,
	-- 把main_id列关联为test_foreign_key_main表的id列
	FOREIGN KEY (main_id) REFERENCES test_foreign_key_main(id)
);

-- 删除外键关联
ALTER TABLE test_foreign_key_ref
DROP FOREIGN KEY test_foreign_key_ref_ibfk_1;

-- 给外键关联指定名字
ALTER TABLE test_foreign_key_ref
ADD CONSTRAINT fk_main_id
FOREIGN KEY (main_id) REFERENCES test_foreign_key_main(id);


-- 给关联了外键的表里插入数据，只能插入main_id在主表里存在的记录，比如1，2
-- 如果插入3，4就会报错
INSERT INTO test_foreign_key_ref (amount, main_id)
VALUES (100.50, 1),
	   (200.75, 2);



几个案例

In [ ]:
-- 不能为不存在的客户创建订单；删除客户前必须先处理其订单（RESTRICT）。
CREATE TABLE customers(
	id INT PRIMARY KEY, 
	name VARCHAR(100));
CREATE TABLE orders(
  id INT PRIMARY KEY,
  customer_id INT,
  FOREIGN KEY (customer_id) REFERENCES customers(id) ON DELETE RESTRICT
);


-- 删除文章时自动删除对应评论
CREATE TABLE posts(id INT PRIMARY KEY, title VARCHAR(200));
CREATE TABLE comments(id INT PRIMARY KEY, post_id INT,
  FOREIGN KEY (post_id) REFERENCES posts(id) ON DELETE CASCADE
);

# 拼接join
组合两个表成一个表

In [ ]:
-- 先创建一个表
CREATE TABLE test_join_a(
	id INT PRIMARY KEY,
	value_a VARCHAR(50)
);

-- 插入些数据
INSERT INTO test_join_a (id, value_a)
VALUES (1, 'A1'),
	   (2, 'A2'),
	   (3, 'A3');

-- 创建另一个表
CREATE TABLE test_join_b(
	id INT PRIMARY KEY,
	value_b VARCHAR(50)
);
-- 插入些数据
INSERT INTO test_join_b (id, value_b)
VALUES (2, 'B2'),
	   (3, 'B3'),
	   (4, 'B4');

-- 接下来做个内连接 inner join
-- 会取两个表都有的内容，比如两个表中id为2和3的记录
SELECT test_join_a.id, value_a, value_b
FROM test_join_a INNER JOIN test_join_b
ON test_join_a.id = test_join_b.id;

-- 做个左连接 left join
-- 会取左表的所有记录，右表有匹配的就取匹配的记录，没有匹配的就显示NULL
SELECT test_join_a.id, value_a, value_b
FROM test_join_a LEFT JOIN test_join_b
ON test_join_a.id = test_join_b.id;


-- 做个右连接 right join
-- 会取右表的所有记录，左表有匹配的就取匹配的记录，没有匹配的就显示NULL
SELECT test_join_a.id, value_a, value_b
FROM test_join_a RIGHT JOIN test_join_b
ON test_join_a.id = test_join_b.id;

# 函数function
自带了很多函数，详见 https://dev.mysql.com/doc/refman/9.5/en/function-resolution.html

In [ ]:
-- 计数
SELECT COUNT(id) AS count_id
FROM test_join_a;

-- 取最大值
SELECT MAX(id) AS max_id
FROM test_join_a;

-- 取平均值
SELECT AVG(id) AS avg_id
FROM test_join_a;

-- 合并
SELECT CONCAT(id,value_a) AS merged
FROM test_join_a;

# 逻辑运算 logical operator

In [ ]:
SELECT	*
FROM test_join_a
WHERE id <= 3 AND NOT value_a = 'A2';

# 通配符 wild card


In [ ]:
-- 插入些数据
INSERT INTO test_join_a (id, value_a)
VALUES (10, 'A1b'),
	   (20, 'A2b'),
	   (30, 'A3b');

-- % 表示任意数量的任意字符
SELECT * FROM test_join_a
WHERE value_a LIKE "%b";


-- _ 表示一个任意字符
SELECT * FROM test_join_a
WHERE value_a LIKE "__b";

# 按条件排序 order by clause

In [ ]:
-- 
SELECT * FROM test_join_a
-- 默认是升序排列ascending
ORDER BY id ;
-- 也可以降序排列descending
ORDER BY id DESC;

# 限制limit clause

In [ ]:
SELECT * FROM test_join_a
LIMIT 2;
-- 加上排序再取限制

SELECT * FROM test_join_a
ORDER BY id DESC LIMIT 2;
-- 设置开始位置，取1条记录
SELECT * FROM test_join_a
LIMIT 2,1;

# 联合运算 union
union 用来把多个查询的结果**纵向合并**在一起，要求每个查询的列数、列类型顺序一致，常见场景：
- 把同结构的分区表、分表的结果拼成一个列表
- 把历史表与当前表的同类数据合并查询
- 用不同条件拆开的查询再合并输出
- 

In [2]:
SELECT * FROM test_join_a
UNION 
SELECT * FROM test_join_b;

-- union只能组合相同列数的表，下面这两个不同列数的表就没法组合
SELECT * FROM test_foreign_key_main
UNION 
SELECT * FROM test_join_b;


SyntaxError: invalid character '，' (U+FF0C) (4194126132.py, line 5)

# 自连接 self join（没看懂
自连接把同一个表当作两个别名来连接，适合处理父子关系、行间比较或生成两两组合。

In [ ]:
-- 简化示例 A：员工 - 经理（层级关系）
DROP TABLE IF EXISTS test_employee;
CREATE TABLE test_employee(id INT PRIMARY KEY, name VARCHAR(50), manager_id INT);
INSERT INTO test_employee VALUES (1,'Alice',2),(2,'Bob',NULL),(3,'Carol',2);
SELECT e.name AS employee, m.name AS manager
FROM test_employee e
LEFT JOIN test_employee m ON e.manager_id = m.id;

-- 简化示例 B：查找重复 email（排除自身）
DROP TABLE IF EXISTS users_test;
CREATE TABLE users_test (id INT PRIMARY KEY, email VARCHAR(100));
INSERT INTO users_test VALUES (1,'a@x.com'),(2,'b@x.com'),(3,'a@x.com');
SELECT a.id AS id1, b.id AS id2, a.email
FROM users_test a
JOIN users_test b ON a.email = b.email AND a.id <> b.id;

-- 简化示例 C：商品两两组合（不重复对）
DROP TABLE IF EXISTS products_test;
CREATE TABLE products_test(id INT PRIMARY KEY, name VARCHAR(50));
INSERT INTO products_test VALUES (1,'P1'),(2,'P2'),(3,'P3');
SELECT a.id AS prod_a, b.id AS prod_b, a.name AS name_a, b.name AS name_b
FROM products_test a
JOIN products_test b ON a.id < b.id;

# 视图 view
视图（VIEW）是一个命名的 SELECT 查询，表现为一个“虚拟表”。它不单独存储数据，而是基于基础表的查询结果动态生成。常见用途：
- **简化复杂查询**：把多表或复杂逻辑封装为一个视图，使用时直接查询视图即可
- **提高重用与可读性**：多个地方复用同一查询逻辑
- **限制访问（安全）**：可以只暴露部分列或行，控制用户能看到的数据
**注意**：视图是基于基础表的，基础表的变化会反映在视图中；并非所有视图都允许更新（部分只读）。

In [ ]:
-- 示例：创建视图并演示视图会动态反映基础表的变化
DROP VIEW IF EXISTS test_view;
CREATE VIEW test_view AS
SELECT id, value_a
FROM test_join_a;

-- 从视图查询（等同于对基础表的查询）
SELECT * FROM test_view;

-- 向基础表插入新数据，视图会反映该变化（使用较大 id 避免冲突）
INSERT INTO test_join_a (id, value_a) VALUES (99, 'A99');
SELECT * FROM test_view;

-- 清理：删除视图并移除示例数据
DROP VIEW IF EXISTS test_view;
DELETE FROM test_join_a WHERE id = 99;

# 索引 index
索引用于加速基于索引列的查询，能显著减少全表扫描，提高 SELECT、JOIN、ORDER BY 速度
- 加快读，但是影响写

详见：https://forum.beginner.center/t/topic/2583

In [ ]:
-- 示例：在 email 列上创建索引并查看查询计划
DROP TABLE IF EXISTS test_index_people;
CREATE TABLE test_index_people (
  id INT PRIMARY KEY AUTO_INCREMENT,
  name VARCHAR(100),
  email VARCHAR(100),
  age INT
);
-- 插入少量示例数据（实际对比建议插入大量数据观察差异）
INSERT INTO test_index_people (name,email,age) VALUES
  ('Alice','a@x.com',30),
  ('Bob','b@x.com',25),
  ('Carol','a@x.com',28);
-- 创建索引
CREATE INDEX idx_email ON test_index_people (email);
SELECT * FROM test_index_people WHERE email = 'a@x.com';


SyntaxError: invalid character '：' (U+FF1A) (2918147388.py, line 1)